Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [1]:
import numpy as np
import pandas as pd
#pd.options.mode.chained_assignment = None
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Generating data to benchmark

In [2]:
%%time
import pickle
import time
### Configuration ###########

#number of datasets
n = 1000

# True coefficients
c = [0.8, #alpha heinz41
     0.2, #alpha heinz32 
     -0.5, #alpha heinz28
     0.9, #mu    display
     1.1, #mu    feat
     -0.5, #mu    price
     0.1, #sigma display 
     0.5, #sigma feat 
     0.25 #sigma price 
    ]

seed = 1234

#############################

np.random.seed(seed)
#brand choices
choices = ["heinz41", "heinz32", "heinz28", "hunts32"]

#generating the beta per household

def drawbeta(household):
    return c[3:6] + (household * np.absolute(c[6:9]))
           #mu       #std        # abs(sigma)

beta = np.apply_along_axis(drawbeta, 1, np.random.randn(300, 3, n))

#defining the choice making process
def makechoice(row, index):
    utilities = np.zeros(4)
    #calculate utility for each brand
    for k in range(4):
        #get the x observation
        x = [row[1+k], row[5+k], row[9+k]]

        #start with the base prefrence
        utility = 0; #start with alpha
        if k < 3: utility = c[k]
        
        #calulate the indiviual utility
        utility += np.dot(x, beta[row.id-1, :, index]) + np.random.gumbel()
        
        #add the error of extreme value???
        
        #save the utility
        utilities[k] = utility

    #pick and set the highest utility brand
    row.choiceindex = np.argmax(utilities)
    row.choice = choices[row.choiceindex]
    return row


#generating the dataset with the dgp
data = [df.copy().apply(makechoice, axis=1, index=r) for r in range(n)]

#organsing the dataset
dataset = {
    "n":n,
    "c":c,
    "seed": seed,
    "beta": beta,
    "data": data  
}

#saving the dataset
pickle_out = open("datasets/n "+str(n)+" - seed "+str(seed)+" - time "+time.strftime("%Y-%m-%d-%H%M%S")+".pickle","wb")
pickle.dump(dataset, pickle_out)
pickle_out.close()


CPU times: user 15min 15s, sys: 1.35 s, total: 15min 16s
Wall time: 15min 15s


In [7]:
#saving the dataset
import pickle
pickle_out = open("x3.pkl","wb")
pickle.dump([1], pickle_out)
pickle.dump("hi", pickle_out)
# pickle_out.close()


In [12]:
import pickle
pkl = open("x3.pkl","rb")
pickle.load(pkl)

[1]

In [16]:
dataset['data'][0].head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz41,0
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz41,0
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz41,0
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Testing importing a dataset

In [15]:
import pickle
example_dict = pickle.load(open("datasets/n 1 - seed 1234 - time 2019-06-17-171420.pickle","rb"))
example_dict['data'][0]

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz41,0
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz41,0
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz41,0
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz28,2
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz28,2
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,heinz28,2
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,heinz28,2
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,heinz28,2
